# 1. Load Data

In [1]:
import pandas as pd
import geopandas as gpd
import datetime
from ast import literal_eval


/Users/xuanedx1/opt/anaconda3/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
mobility = pd.read_csv("sg_ca_data.csv", parse_dates=['date_range_start'], dtype={0: str, 8: str, 9:str})

In [3]:
mobility['bucketed_distance_travelled'] = mobility['bucketed_distance_travelled'].apply(literal_eval)

In [4]:
mobility.head()

,origin_census_block_group,date_range_start,device_count,distance_traveled_from_home,bucketed_distance_travelled,completely_home_device_count,median_percentage_time_home,mean_distance_traveled_from_home,county_fips,cbg_fips
0,060014075002,2019-12-21 08:00:00+00:00,71,1598.0,"{'16001-50000': 4, '0': 27, '>50000': 5, '2001...",28,95,22833.0,06001,060014075002
1,060190057022,2019-12-21 08:00:00+00:00,80,3227.0,"{'16001-50000': 5, '0': 26, '>50000': 1, '2001...",26,88,4215.0,06019,060190057022
2,060210101002,2019-12-21 08:00:00+00:00,101,2405.0,"{'16001-50000': 29, '0': 27, '>50000': 15, '20...",30,88,19155.0,06021,060210101002
3,060290060071,2019-12-21 08:00:00+00:00,81,4593.0,"{'16001-50000': 3, '0': 23, '>50000': 22, '200...",21,81,21754.0,06029,060290060071
4,060310016011,2019-12-21 08:00:00+00:00,82,4056.0,"{'16001-50000': 17, '0': 23, '>50000': 14, '20...",23,79,12784.0,06031,060310016011


In [5]:
mobility.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7294981 entries, 0 to 7294980
Data columns (total 10 columns):
 #   Column                            Dtype              
---  ------                            -----              
 0   origin_census_block_group         object             
 1   date_range_start                  datetime64[ns, UTC]
 2   device_count                      int64              
 3   distance_traveled_from_home       float64            
 4   bucketed_distance_travelled       object             
 5   completely_home_device_count      int64              
 6   median_percentage_time_home       int64              
 7   mean_distance_traveled_from_home  float64            
 8   county_fips                       object             
 9   cbg_fips                          object             
dtypes: datetime64[ns, UTC](1), float64(2), int64(3), object(4)
memory usage: 556.6+ MB


# 2. Data Preprocessing

In [ ]:
# weighted average for bucketed_distance_travelled (by lower bound in distance bin)
distance = [33000, 0, 50000, 5000, 500, 1500, 12000]
mobility['weighted_avg_bucketed_distance_travelled'] = mobility["bucketed_distance_travelled"].apply(lambda x: sum([a*b for a,b in zip(list(x.values()), distance)])/sum(x.values()))

In [ ]:
mobility.head()


In [ ]:
# aggregate data by week and county level
mobility['date_range_start'] = mobility["date_range_start"].apply(lambda x: datetime.datetime(year=x.year, month=x.month, day=x.day))

In [ ]:
# aggregate data by week and county level
grouped_mobility = mobility.groupby('county_fips').resample('W', on='date_range_start').sum()

In [ ]:
grouped_mobility

In [ ]:
# reset index
grouped_mobility = grouped_mobility.reset_index()
grouped_mobility['county_fips'] = grouped_mobility['county_fips'].str[2:]

In [ ]:
# merge mobility data with geo data
geodata = gpd.read_file("CA_Counties/CA_Counties_TIGER2016.shp")


In [ ]:
geodata.head()


In [ ]:
data = grouped_mobility[['county_fips','date_range_start','weighted_avg_bucketed_distance_travelled']].merge(geodata[['COUNTYFP', 'geometry']], how='left', left_on = 'county_fips', right_on='COUNTYFP').sort_values('date_range_start')

In [ ]:
data = data.drop('COUNTYFP', axis=1)

In [ ]:
data

# 3. Exploring spatial structure

In [ ]:
# analysis
import libpysal
from esda.moran import Moran
from esda.moran import Moran_Local
from numpy.random import seed
from libpysal.weights.contiguity import Queen


## Global and Local Moran I 

In [ ]:
data.info()

In [ ]:
idx_df = data.groupby('date_range_start')[['county_fips']].count().cumsum().reset_index()

In [ ]:
# get global moran I and p-values
import warnings
warnings.filterwarnings("ignore", message="Numerical issues were encountered ")

index = idx_df.county_fips.values
pre_idx = -1
moran_G_raw = [0]*len(index)
moran_L_raw = [0]*len(index)

moran_ = [0]*len(index)
moran_G = [0]*len(index)
p_value_G = [0]*len(index)
moran_L = [0]*len(index)
p_value_L = [0]*len(index)

for i, idx in enumerate(index):
    # Generate W from the GeoDataFrame
    week_df = data.iloc[pre_idx+1:idx,:]
    w = Queen.from_dataframe(week_df, idVariable='county_fips')
    # Row-standardization
    w.transform = 'r'
    
    # target variable values
    y = week_df['weighted_avg_bucketed_distance_travelled']
    
    # calculate moran I
    moran = Moran(y, w)
    
    moran_[i] = moran
    moran_G[i] = moran.I
    p_value_G[i] = moran.p_sim
    
    moran_L_raw[i] = Moran_Local(y, w)
    moran_L[i] = Moran_Local(y, w).q
    p_value_L[i] = Moran_Local(y, w).p_sim
    
    # reset left index
    pre_idx = idx

# 4. Visualizing and mapping spatial autocorrelation

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import seaborn as sns
import numpy as np

In [ ]:
# concat data with week
idx_df['global_moran_I'] = moran_G
idx_df['global_moran_I_p'] = p_value_G
idx_df['local_moran_I'] = moran_L
idx_df['local_moran_I_p'] = p_value_L

In [ ]:
idx_df.head()


In [ ]:
idx_df['significance'] = np.where(idx_df['global_moran_I_p'] < 0.05, 'yes', 'no')
idx_df.head()


## Weekly Global Moran's I

In [ ]:
# global moran I v.s. time 
plt.figure(figsize=(10, 8)) # fig size
sns.lineplot(data=idx_df, x='date_range_start', y='global_moran_I', color='black') # lineplot
sns.scatterplot(data=idx_df, x='date_range_start', y='global_moran_I', hue = 'significance', s=80, palette=dict(no='white',yes='black'), edgecolor="black") # add marker
plt.axvline(pd.Timestamp('2020-03-11'), color='r', ls='--') # add vertical line
plt.title("Weekly Global Moran's I", fontsize = 14)
plt.xlabel("Time", fontsize = 12)
plt.ylabel("Global Moran I", fontsize = 12)

**Observations:**
1. There is a decreasing trend in global Moran's I from 03/2020 to 10/2020
2. The 2 greatest and lowest global Moran's I are found to be significant, the rest of the value are insignificant.
3. Overall, the range of Moran's I is small, fluctuating between -0.1 and 0.1
4. The red dash vertical line marked the begining of the pandemic

**Interpretations:**
1. Overall, the spatial autocorrelations of the average traveled distances of mobile devices is not strong, but comparing the Moran's I across the time we could oberseve the impact of pandemic on the spatial relationship of people's mobility.
2. Before and at the begining of the pandemic, the average traveled distances of mobile devices has a relatively positive spatial autocorrelation at a highest of 0.1, indicating people in counties of similar mibility are closer to each other. However, the spatial autocorrelation went down to negative as the pandemic went on, reaching -0.1 in the summer 2020, showing that people in counties of similar mibility were far away. 

## Moran's I Scatterplot for whole dataset

In [ ]:
from splot.esda import moran_scatterplot
from splot.esda import plot_local_autocorrelation

In [ ]:
w = Queen.from_dataframe(data)
moran = Moran(data['weighted_avg_bucketed_distance_travelled'], w)

In [ ]:
fig, ax = moran_scatterplot(moran, aspect_equal=True)
plt.figsize=(30,30)
plt.show()

## Local Moran's I for Week3, 12, 40

In [ ]:
# 3 subplots for Local Moran’s (LISA) for Week 3, Week 12, and Week 40
fig, axs = plt.subplots(1, 3, figsize=(20,15),subplot_kw={'aspect': 'equal'})
moran_scatterplot(moran_L_raw[2], p=0.05, zstandard=False, ax=axs[0]) #week 3
moran_scatterplot(moran_L_raw[11], p=0.05, zstandard=False, ax=axs[1])# week 12
moran_scatterplot(moran_L_raw[39], p=0.05, zstandard=False, ax=axs[2]) # week 40

# set title
axs[0].set_title("Week 3 Local Moran's I Scatterplot ")
axs[1].set_title("Week 12 Local Moran's I Scatterplot ")
axs[2].set_title("Week 40 Local Moran's I Scatterplot ")

**Week 3:**  
   1. 1 hotspot county (red) in High-High local spatial autocorrelation zone(upper right), i.e. high mobility county surrouned by high mobility counties
   2. 4 coldspot county (blue) in Low-Low local spatial autocorrelation zone(lower left), i.e. low mobility county surrouned by low mobility counties
   3. More counties Distributed in Low-High, Low-Low zone, showing many low mobility counties are roughly equally surrounded by high and low mobility counties, 
   i.e. mobility of neighbor counties are dissimilar
   
**Week 12:**
   1. no hotspot, 2 coldspot. 
   2. most counties in Low-Low local spatial autocorrelation zone, i.e. mobility of neighbor counties are highly similar in that all are low.

**Week 40:**
   1. 1 hotspot, 1-2 coldspot. 
   2. most counties in Low-Low local spatial autocorrelation zone, but a hotspot reappear. i.e. most counties has similar low mobility, 1 counties reapprear high spatial autocorrelation in mobility

Overtime, the mobility in California Counties tend to decrease over 2020, however, a few county revive it mobility to high level at the end of the year.  


In [ ]:
!jupyter nbconvert --to pdf --no-input Wenxuan_Zhang_Lab_test.ipynb